<a href="https://colab.research.google.com/github/Alfredchong726/DLI-Group-Assignment/blob/main/DLI_Group_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## First commint to GitHub